In [1]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import pickle as pickle
import gym
import time

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = True # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
  model = pickle.load(open('save2.p', 'rb'))
else:
  model = {}
  model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
  model['W2'] = np.random.randn(H) / np.sqrt(H)
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(np.float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
  if render: env.render()

  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x

  # forward the policy network and sample an action from the returned probability
  aprob, h = policy_forward(x)
  action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

  # record various intermediates (needed later for backprop)
  xs.append(x) # observation
  hs.append(h) # hidden state
  y = 1 if action == 2 else 0 # a "fake label"
  dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

  # step the environment and get new measurements
  observation, reward, done, info = env.step(action)
  reward_sum += reward

  drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

  if done: # an episode finished
    episode_number += 1

    # stack together all inputs, hidden states, action gradients, and rewards for this episode
    epx = np.vstack(xs)
    eph = np.vstack(hs)
    epdlogp = np.vstack(dlogps)
    epr = np.vstack(drs)
    xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
    discounted_epr = discount_rewards(epr)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_epr -= np.mean(discounted_epr)
    discounted_epr /= np.std(discounted_epr)

    epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
    grad = policy_backward(eph, epdlogp)
    for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

    # perform rmsprop parameter update every batch_size episodes
    if episode_number % batch_size == 0:
      for k,v in model.items():
        g = grad_buffer[k] # gradient
        rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
        model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
        grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
    running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
    try: print(time.clock() - start)
    except: print("boo")
    print('resetting env. episode reward total was {}. running mean: {}'.format(reward_sum, running_reward))
    start = time.clock()
    if episode_number % 50 == 0: pickle.dump(model, open('save2.p', 'wb'))
    reward_sum = 0
    observation = env.reset() # reset env
    prev_x = None

    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print(('ep{}: game finished, reward: {}'.format(episode_number, reward)) + ('' if reward == -1 else ' !!!!!!!!'))

/home/hlynur/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


boo
resetting env. episode reward total was -20.0. running mean: -20.0
ep1: game finished, reward: -1.0
14.484034999999995
resetting env. episode reward total was -17.0. running mean: -19.970000000000002
ep2: game finished, reward: -1.0
20.361564
resetting env. episode reward total was -15.0. running mean: -19.9203
ep3: game finished, reward: -1.0
17.236117
resetting env. episode reward total was -18.0. running mean: -19.901097
ep4: game finished, reward: -1.0
14.998988000000011
resetting env. episode reward total was -19.0. running mean: -19.89208603
ep5: game finished, reward: -1.0
16.269700999999998
resetting env. episode reward total was -16.0. running mean: -19.853165169700002
ep6: game finished, reward: -1.0
15.105128999999991
resetting env. episode reward total was -18.0. running mean: -19.834633518003002
ep7: game finished, reward: -1.0
18.931928999999997
resetting env. episode reward total was -15.0. running mean: -19.78628718282297
ep8: game finished, reward: -1.0
17.45220399

17.411828000000014
resetting env. episode reward total was -18.0. running mean: -18.932378644536907
ep64: game finished, reward: -1.0
16.593208999999888
resetting env. episode reward total was -19.0. running mean: -18.933054858091538
ep65: game finished, reward: -1.0
15.555650999999898
resetting env. episode reward total was -18.0. running mean: -18.92372430951062
ep66: game finished, reward: -1.0
16.83496100000002
resetting env. episode reward total was -17.0. running mean: -18.904487066415516
ep67: game finished, reward: -1.0
19.82754799999998
resetting env. episode reward total was -17.0. running mean: -18.885442195751363
ep68: game finished, reward: -1.0
21.317289000000073
resetting env. episode reward total was -15.0. running mean: -18.846587773793846
ep69: game finished, reward: -1.0
17.923412999999982
resetting env. episode reward total was -15.0. running mean: -18.808121896055905
ep70: game finished, reward: -1.0
20.76496300000008
resetting env. episode reward total was -17.0. 

22.483009999999922
resetting env. episode reward total was -16.0. running mean: -18.423661590705105
ep126: game finished, reward: -1.0
19.262220999999954
resetting env. episode reward total was -20.0. running mean: -18.439424974798055
ep127: game finished, reward: -1.0
17.7147130000003
resetting env. episode reward total was -14.0. running mean: -18.395030725050074
ep128: game finished, reward: -1.0
22.341511000000082
resetting env. episode reward total was -15.0. running mean: -18.36108041779957
ep129: game finished, reward: -1.0
23.476914000000306
resetting env. episode reward total was -13.0. running mean: -18.307469613621574
ep130: game finished, reward: -1.0
15.706288999999742
resetting env. episode reward total was -21.0. running mean: -18.33439491748536
ep131: game finished, reward: -1.0
18.157388000000083
resetting env. episode reward total was -19.0. running mean: -18.341050968310505
ep132: game finished, reward: -1.0
19.295012000000042
resetting env. episode reward total was 

18.80800999999974
resetting env. episode reward total was -11.0. running mean: -17.88526745039768
ep188: game finished, reward: -1.0
18.348696000000018
resetting env. episode reward total was -19.0. running mean: -17.896414775893707
ep189: game finished, reward: -1.0
15.872961999999916
resetting env. episode reward total was -17.0. running mean: -17.887450628134772
ep190: game finished, reward: -1.0
22.625714000000244
resetting env. episode reward total was -15.0. running mean: -17.858576121853424
ep191: game finished, reward: -1.0
18.503211000000192
resetting env. episode reward total was -17.0. running mean: -17.84999036063489
ep192: game finished, reward: -1.0
17.112307999999757
resetting env. episode reward total was -17.0. running mean: -17.841490457028545
ep193: game finished, reward: -1.0
14.618379000000004
resetting env. episode reward total was -19.0. running mean: -17.85307555245826
ep194: game finished, reward: -1.0
15.737039999999979
resetting env. episode reward total was 

...

19.000911999999516
resetting env. episode reward total was -17.0. running mean: -17.55078243937671
ep312: game finished, reward: -1.0
16.676277000000482
resetting env. episode reward total was -19.0. running mean: -17.565274614982943
ep313: game finished, reward: -1.0
16.840368999999555
resetting env. episode reward total was -19.0. running mean: -17.579621868833115
ep314: game finished, reward: -1.0
18.789828999999372
resetting env. episode reward total was -16.0. running mean: -17.563825650144786
ep315: game finished, reward: -1.0
18.429478000000017
resetting env. episode reward total was -18.0. running mean: -17.56818739364334
ep316: game finished, reward: -1.0
17.2625850000004
resetting env. episode reward total was -18.0. running mean: -17.572505519706905
ep317: game finished, reward: -1.0
21.65605400000004
resetting env. episode reward total was -17.0. running mean: -17.566780464509836
ep318: game finished, reward: -1.0
16.033110999999735
resetting env. episode reward total was -

19.58349399999952
resetting env. episode reward total was -19.0. running mean: -17.80869438412832
ep373: game finished, reward: -1.0
15.727280999999493
resetting env. episode reward total was -21.0. running mean: -17.840607440287037
ep374: game finished, reward: -1.0
23.075326000000132
resetting env. episode reward total was -19.0. running mean: -17.852201365884167
ep375: game finished, reward: -1.0
17.36208600000009
resetting env. episode reward total was -19.0. running mean: -17.863679352225326
ep376: game finished, reward: -1.0
18.358663999999408
resetting env. episode reward total was -12.0. running mean: -17.80504255870307
ep377: game finished, reward: -1.0
17.241244000000734
resetting env. episode reward total was -18.0. running mean: -17.80699213311604
ep378: game finished, reward: -1.0
19.405033000000003
resetting env. episode reward total was -16.0. running mean: -17.78892221178488
ep379: game finished, reward: -1.0
18.802036000000044
resetting env. episode reward total was -1

17.428812999998627
resetting env. episode reward total was -18.0. running mean: -17.82565220757563
ep435: game finished, reward: -1.0
14.84969699999965
resetting env. episode reward total was -16.0. running mean: -17.807395685499873
ep436: game finished, reward: -1.0
20.296304999999847
resetting env. episode reward total was -19.0. running mean: -17.819321728644876
ep437: game finished, reward: -1.0
15.656087999999727
resetting env. episode reward total was -20.0. running mean: -17.841128511358427
ep438: game finished, reward: -1.0
17.50057900000138
resetting env. episode reward total was -17.0. running mean: -17.832717226244846
ep439: game finished, reward: -1.0
25.122754999998506
resetting env. episode reward total was -18.0. running mean: -17.834390053982396
ep440: game finished, reward: -1.0
20.873718000000736
resetting env. episode reward total was -19.0. running mean: -17.846046153442572
ep441: game finished, reward: -1.0
17.585934999999154
resetting env. episode reward total was

18.7258499999989
resetting env. episode reward total was -19.0. running mean: -17.8256444093476
ep497: game finished, reward: -1.0
19.001162000000477
resetting env. episode reward total was -21.0. running mean: -17.857387965254123
ep498: game finished, reward: -1.0
18.542186999999103
resetting env. episode reward total was -19.0. running mean: -17.86881408560158
ep499: game finished, reward: -1.0
21.83866400000079
resetting env. episode reward total was -17.0. running mean: -17.860125944745565
ep500: game finished, reward: -1.0
25.48303499999929
resetting env. episode reward total was -13.0. running mean: -17.81152468529811
ep501: game finished, reward: -1.0
22.522923000000446
resetting env. episode reward total was -15.0. running mean: -17.783409438445126
ep502: game finished, reward: -1.0
24.630150999999387
resetting env. episode reward total was -17.0. running mean: -17.775575344060677
ep503: game finished, reward: -1.0
19.253445000000283
resetting env. episode reward total was -21.

KeyboardInterrupt: 